# Prepare ESA CCI luc data

In [ ]:
# Libraries
import os, time, shutil, rioxarray
import numpy as np
import pandas as pd
import xarray as xr
from scipy import ndimage
import matplotlib.pyplot as plt

In [ ]:
# Directories
dir_data =  '../data/'
dir01 = '../paper_deficit/output/01_prep/'

---

### Pre-processing

In [ ]:
# Libraries
from dask_jobqueue import SLURMCluster
from dask.distributed import Client
import dask

# Initialize dask
cluster = SLURMCluster(
    queue='compute',                      # SLURM queue to use
    cores=32,                             # Number of CPU cores per job
    memory='256 GB',                      # Memory per job
    account='bm0891',                     # Account allocation
    interface="ib0",                      # Network interface for communication
    walltime='06:00:00',                  # Maximum runtime per job
    local_directory='../dask/',           # Directory for local storage
    job_extra_directives=[                # Additional SLURM directives for logging
        '-o ../dask/LOG_worker_%j.o',     # Output log
        '-e ../dask/LOG_worker_%j.e'      # Error log
    ]
)

# Scale dask cluster
cluster.scale(jobs=10)

# Configurate dashboard url
dask.config.config.get('distributed').get('dashboard').update(
    {'link': '{JUPYTERHUB_SERVICE_PREFIX}/proxy/{port}/status'}
)

# Create client
client = Client(cluster)

client

In [ ]:
# Function to trim memory of workers
def trim_memory() -> int:
    import ctypes
    libc = ctypes.CDLL("libc.so.6")
    return libc.malloc_trim(0)

In [ ]:
def prep_esacci():
    
    """Prepare stable ESA-CCI grid cell data by processing land cover classes."""
    
    # Step 1: Load ESA-CCI land cover datasets
    esacci = xr.open_mfdataset(dir_data + 'esa_luc/*.nc', combine='by_coords') \
        .chunk(dict(lat=5000, lon=5000, time=1)) \
        .lccs_class

    # Step 2: Convert the time dimension to just the year
    esacci['time'] = pd.DatetimeIndex(esacci.time.data).year

    # Step 3: Process each land cover type (based on 'flag_values' attribute)
    for i in esacci.attrs['flag_values']:
        # Create a new dataset marking cells where the specific land cover type (i) is present
        esacci2 = xr.Dataset()
        esacci2['esacci_' + str(i)] = xr.where(esacci == i, True, False)
        
        # Step 4: Save the new dataset for the specific land cover type as a Zarr file
        esacci2.to_zarr(
            dir01 + 'esacci_prep/esacci_prep_' + str(i) + '.zarr', mode='w')

        # Step 5: Reload the saved dataset and re-chunk it
        esacci2 = xr.open_zarr(
            dir01 + 'esacci_prep/esacci_prep_' + str(i) + '.zarr') \
            .chunk(dict(lat=5000, lon=5000, time=-1))

        # Step 6: Identify cells without land-use change (LUC) over the years (no change = stable land cover)
        # If the sum over time equals 29 (indicating presence for all 29 years), it's a stable land cover
        xr.where(esacci2.sum('time') == 29, True, False) \
            .to_zarr(dir01 + 'esacci_prep/esacci_prep_sum_' + str(i) + '.zarr', 
                     mode='w')
        
        # Release memory after processing each land cover type
        client.run(trim_memory)

    # Step 7: Open all the prepared land cover stability datasets
    esacci_prep_sum = xr.open_mfdataset(
        dir01 + 'esacci_prep/esacci_prep_sum_*.zarr', engine='zarr')

    # Step 8: Identify stable grid cells for specific land cover types
    # Stable crop grid cells (classes 10, 11, 12, 20)
    esacci_crop = (
        esacci_prep_sum.esacci_10 + esacci_prep_sum.esacci_11 +
        esacci_prep_sum.esacci_12 + esacci_prep_sum.esacci_20
    )
    
    # Stable pasture grid cells (class 130)
    esacci_pasture = esacci_prep_sum.esacci_130
    
    # Stable forest and shrubland grid cells (many classes combined)
    esacci_fso = (
        esacci_prep_sum.esacci_50 + esacci_prep_sum.esacci_60 +
        esacci_prep_sum.esacci_61 + esacci_prep_sum.esacci_62 +
        esacci_prep_sum.esacci_70 + esacci_prep_sum.esacci_71 +
        esacci_prep_sum.esacci_72 + esacci_prep_sum.esacci_80 +
        esacci_prep_sum.esacci_81 + esacci_prep_sum.esacci_82 +
        esacci_prep_sum.esacci_90 + esacci_prep_sum.esacci_100 +
        esacci_prep_sum.esacci_110 + esacci_prep_sum.esacci_120 +
        esacci_prep_sum.esacci_121 + esacci_prep_sum.esacci_122 +
        esacci_prep_sum.esacci_140 + esacci_prep_sum.esacci_150 +
        esacci_prep_sum.esacci_151 + esacci_prep_sum.esacci_152 +
        esacci_prep_sum.esacci_153 + esacci_prep_sum.esacci_160 +
        esacci_prep_sum.esacci_170 + esacci_prep_sum.esacci_180 +
        esacci_prep_sum.esacci_190 + esacci_prep_sum.esacci_200 +
        esacci_prep_sum.esacci_201 + esacci_prep_sum.esacci_202
    )

    # Step 9: Create a dataset containing stable grid cells for crops, pasture, and forest/shrubland (FSO)
    esacci_prep_stable = xr.Dataset({
        'esacci_stable_crop': esacci_crop,
        'esacci_stable_pasture': esacci_pasture,
        'esacci_stable_fso': esacci_fso
    })

    # Step 10: Export the stable grid cell dataset as a Zarr file
    esacci_prep_stable.to_zarr(dir01 + 'esacci_prep_stable.zarr', mode='w')

    # Step 11: Reload the stable grid cell dataset
    esacci_prep_stable = xr.open_zarr(dir01 + 'esacci_prep_stable.zarr')

    # Step 12: Export each stable grid cell variable as GeoTIFF
    for var in esacci_prep_stable.data_vars:
        esacci_prep_stable[var] \
            .rename(dict(lat='y', lon='x')) \
            .astype('float32') \
            .rio.to_raster(dir01 + var + '.tif')

        # Release memory after each export
        client.run(trim_memory)

    # Step 13: Clean up intermediate files and directories
    shutil.rmtree(dir01 + 'esacci_prep')  # Remove temporary directories
    shutil.rmtree(dir01 + 'esacci_prep_stable.zarr')  # Remove Zarr directory after TIFF export


%time prep_esacci()

In [ ]:
# Close dask cluster
cluster.close()

In [ ]:
# Wait for 60s for dask client to completely disconnect
time.sleep(60)

---

### Regridding

In [ ]:
# Import regridding function
from regrid_high_res_v1_01 import regrid_high_res, prep_tif

In [ ]:
def regrid_da(f_source, dir_target, dir_source, dir_out, 
              size_tiles, fill_value=None, olap=1):  
    """Regrid large xarray dataarrays.

    Args:
        f_source (str): The filename (without extension) of the source .tif file to be regridded.
        dir_target (str): Directory containing target grid .tif file.
        dir_source (str): Directory containing the the source  .tif file.
        dir_out (str): Directory to store the output and intermediate files.
        size_tiles (int): Size of the regridding tiles in degrees.
        fill_value (float, optional): Fill value to use in the regridding process. Defaults to None.
        olap (int, optional): Overlap size in degrees for regridding tiles. Defaults to 1.
        
    Returns:
        xarray.Dataset: The combined dataset after regridding.
    """
    # Prepare the target and source data arrays from TIFF files
    da_target = prep_tif(dir_target + 'target_grid.tif', 'target_grid')
    da_source = prep_tif(dir_source + f_source + '.tif', f_source)
    # Regridd source array to target grid
    regrid_high_res(da_target, da_source, dir_out,
                    account='bm0891', partition='compute',
                    size_tiles=size_tiles, olap=olap, fill_value = fill_value,
                    type_export='zarr', del_interm=False)

In [ ]:
# regridding
list_esacci = ['esacci_stable_crop', 'esacci_stable_pasture', 
               'esacci_stable_fso']

for i in list_esacci:
    print(i)
    %time regrid_da(i, dir01, dir01, dir01, 30, 127, 0.1)

---

### Fill nan

In [ ]:
def fill_nans(var, dir_out):
    """
    Fills NaN values in the specified variable's dataset using the nearest valid 
    data, applies a land mask, and exports the result to a new Zarr dataset.

    Args:
        var (str): Name of the variable to process (e.g., 'temperature', 'precipitation').
        dir_out (str): Directory where prepared data is stored and the filled dataset will be exported.

    Returns:
        None
    """

    def fill_nans_array(data, invalid):
        """
        Replace invalid (NaN) data cells by the value of the nearest valid data 
        cell.
        """
        ind = ndimage.distance_transform_edt(invalid,
                                             return_distances=False,
                                             return_indices=True)
        return data[tuple(ind)]

    # Paths for input and output
    land_mask_path = os.path.join(dir_out, 'ds_prep_copernicus_land_mask.zarr')
    var_data_path = os.path.join(dir_out, f'ds_regridded_{var}.zarr')
    output_path = os.path.join(dir_out, f'ds_prep_{var}.zarr')

    # Read land mask data
    da_land = xr.open_zarr(land_mask_path) \
                .chunk(dict(lat=5000, lon=5000)) \
                .copernicus_land_mask \
                .compute()

    # Read variable data
    da_var = xr.open_zarr(var_data_path)['regridded_' + var]

    # Fill nan using function fill_nans_array
    # If there are no NaNs, skip filling process
    if not da_var.isnull().any():
        da_fill = da_var.values  # No filling required
    else:
        da_fill = fill_nans_array(da_var.values, da_var.isnull().values)

    # Create a new Dataset with filled data
    ds_filled = xr.Dataset(dict(lat = da_var.lat, lon=da_var.lon))
    ds_filled[var] = (('lat', 'lon'), da_fill)

    # Apply land mask to the filled data
    ds_filled = ds_filled.where(da_land)

    # Export the filled dataset to Zarr format
    ds_filled.chunk(dict(lat=5000, lon=5000)) \
             .to_zarr(output_path, mode='w')

In [ ]:
%%time
for i in ['esacci_stable_crop', 'esacci_stable_pasture', 
          'esacci_stable_fso']:
    %time fill_nans(i, dir01)

---

### Check

In [ ]:
# Plot to check
for i in ['esacci_stable_crop', 'esacci_stable_pasture', 
          'esacci_stable_fso']:
    fig, ax = plt.subplots(figsize=(10, 5), ncols=1, nrows=1)
    xr.open_zarr(dir01 + 'ds_prep_' + i + '.zarr')[i] \
        .plot.imshow(ax=ax)
    ax.set_title(i)